In [4]:
# это мы уже делали в предыдущей тетрадке
import sys
import pandas as pd
import scipy
import numpy as np
from os import path
from scipy.sparse import csr_matrix

data_dir = "/home/ubuntu/data/movielens/ml-latest-small"
def read_csv(filename: str):
    data = pd.read_csv(path.join(data_dir, filename + ".csv"))
    return data

ratings = read_csv("ratings")
movies = read_csv("movies")
ratings["movie_id"] = ratings["movieId"].astype("category").cat.codes.copy() + 1

def sparse_info(sparse_matrix: csr_matrix):
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

last_movie_id = ratings["movie_id"].max()
last_user_id = ratings["userId"].max()

In [5]:
mean_rating = ratings["rating"].mean()
print("самое простое предсказание оценки - это средняя оценка")
print(mean_rating)

самое простое предсказание оценки - это средняя оценка
3.54360825567


In [6]:
user_x_item = ratings[["userId", "movie_id"]].as_matrix()
# здесь уже на пересечении строк и столбцов матрицы стоят не единички
# а отклонение оценки от средней оценки
user_item_matrix = csr_matrix(
    (
        (ratings["rating"] - mean_rating).tolist(),
        (
            [pair[0] for pair in user_x_item],
            [pair[1] for pair in user_x_item],
        )
    ),
    shape=(last_user_id + 1, last_movie_id + 1),
    dtype=np.float32
)

sparse_info(user_item_matrix)

Размерности матрицы: (672, 9067)
Ненулевых элементов в матрице: 100004
Доля ненулевых элементов: 0.01641286822438251
Среднее значение ненулевых элементов: -8.544580154534742e-09
Максимальное значение ненулевых элементов: 1.45639169216156
Минимальное значение ненулевых элементов: -3.0436081886291504


In [17]:
# теперь найдём матрицу схожести между фильмами
from sklearn.preprocessing import normalize, binarize

def get_cosine_similarity_matrix(user_item_matrix):
    # матрицы этого типа быстрее умножаются
    user_item_csr = user_item_matrix.tocsr()
    # нормализация и последующее умножение эквивалентно нахождению косинуса между столбцами матрицы
    user_item_normalized = normalize(user_item_csr, norm='l2', axis=0)
    return user_item_normalized.T.dot(user_item_normalized)

similarity_matrix = get_cosine_similarity_matrix(user_item_matrix)
sparse_info(similarity_matrix)

Размерности матрицы: (9067, 9067)
Ненулевых элементов в матрице: 21983224
Доля ненулевых элементов: 0.26740169371818234
Среднее значение ненулевых элементов: 0.04221750795841217
Максимальное значение ненулевых элементов: 1.0000016689300537
Минимальное значение ненулевых элементов: -1.0


In [18]:
from scipy.sparse import vstack, csr_matrix

# оставляем только top k схожих элементов (k ближайших соседей)
def get_top_k_in_a_row(similarity_matrix, row, k, no_diagonal):
    lil_row = similarity_matrix[row]
    # каждый фильм очень сильно схож сам с собой, но это бесполезная информация
    if no_diagonal:
        lil_row[0, row] = 0
    csr_row = lil_row.tocsr()
    csr_row.data[csr_row.data.argsort()[:-k]] = 0
    csr_row.eliminate_zeros()
    return csr_row

def get_top_k(similarity_matrix, k, no_diagonal=True):
    # с матрицами этого типа удобнее всего работать построчно
    lil_similarity = similarity_matrix.tolil()
    top_k_similarity_matrix = get_top_k_in_a_row(lil_similarity, 0, k, no_diagonal)
    for row in range(1, lil_similarity.shape[0]):
        if len(lil_similarity.rows[row]) > 0:
            csr_row = get_top_k_in_a_row(lil_similarity, row, k, no_diagonal)
            top_k_similarity_matrix = vstack([top_k_similarity_matrix, csr_row])
    return top_k_similarity_matrix

In [19]:
top_k_similarity_matrix = get_top_k(similarity_matrix, 5)
del similarity_matrix
print("ненулевых элементов стало гораздо меньше")
sparse_info(top_k_similarity_matrix)

ненулевых элементов стало гораздо меньше
Размерности матрицы: (9067, 9067)
Ненулевых элементов в матрице: 45330
Доля ненулевых элементов: 0.0005513894948368449
Среднее значение ненулевых элементов: 0.7609193921089172
Максимальное значение ненулевых элементов: 1.0
Минимальное значение ненулевых элементов: 0.0029404032975435257


In [21]:
# выделяем обучающую и тестовую выборки
from scipy.sparse import coo_matrix

np.random.seed(0)
train_percent = 0.1
user_item_matrix = user_item_matrix.tocoo()
train_split = np.random.choice(
    range(user_item_matrix.nnz),
    int(user_item_matrix.nnz * train_percent),
    replace=False
)
test_split = list(set(range(user_item_matrix.nnz)) - set(train_split))
train_matrix = coo_matrix(
    (
        user_item_matrix.data[train_split],
        (user_item_matrix.row[train_split], user_item_matrix.col[train_split])
    ),
    shape=user_item_matrix.shape
)
test_matrix = coo_matrix(
    (
        user_item_matrix.data[test_split],
        (user_item_matrix.row[test_split], user_item_matrix.col[test_split])
    ),
    shape=user_item_matrix.shape
)
print("Размер обучающей выборки:", train_matrix.nnz)
print("Размер тестовой выборки:", test_matrix.nnz)

Размер обучающей выборки: 10000
Размер тестовой выборки: 90004


In [22]:
def RMSE(matrix1: coo_matrix, matrix2: coo_matrix):
    return np.sqrt(((matrix1 - matrix2).data ** 2).mean())

mean_rating_prediction = 0 * test_matrix
print("точность нашего baseline")
print(RMSE(mean_rating_prediction, test_matrix))

точность нашего baseline
1.0579


In [23]:
# нахождение предсказаний эквивалентно умножению нормированной матрицы схожести на вектор оценок пользователя
normalized_similarity = normalize(top_k_similarity_matrix.tocsr(), norm="l1", axis=0)
raw_predictions = train_matrix.dot(normalized_similarity).tocoo()
print("получили предсказаний всего")
print(len(raw_predictions.data))

получили предсказаний всего
45330


In [24]:
filtered_predictions = binarize(test_matrix).multiply(raw_predictions)
print("оставили предсказаний для теста")
print(len(filtered_predictions.data))

оставили предсказаний для теста
5198


In [25]:
print(RMSE(filtered_predictions, test_matrix))

1.04981
